<a href="https://colab.research.google.com/github/amanmehra-23/RP_RecommenderSystem/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1) Install dependencies (run once)
!pip install faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
# 2) Imports & MODEL path
import os
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

MODELS_DIR = "/content/"   # adjust if you uploaded elsewhere

# 3) Load your data, index, embeddings, and SBERT model
df = pd.read_pickle(os.path.join(MODELS_DIR, "dataset.pkl"))
embeddings = np.load(os.path.join(MODELS_DIR, "embeddings.npy"))
index = faiss.read_index(os.path.join(MODELS_DIR, "faiss_index.bin"))
sbert = SentenceTransformer("all-mpnet-base-v2", device="cpu")

# 4) Helper: build combo, embed, search, and return a DataFrame
def get_recommendations(terms: str, title: str, abstract: str, k: int = 5):
    # build query string
    combo = "  ||  ".join([
        terms.replace(",", " "),
        title,
        abstract
    ]).strip()
    # embed + normalize
    q_emb = sbert.encode([combo], convert_to_tensor=False)
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)
    # FAISS search
    D, I = index.search(q_emb.astype(np.float32), k)
    # assemble results
    recs = []
    for score, idx in zip(D[0], I[0]):
        recs.append({
            "score": float(score),
            "terms": df.at[idx, "terms"],
            "title": df.at[idx, "titles"],
            "abstract": df.at[idx, "abstracts"][:200] + "…"  # truncate
        })
    return pd.DataFrame(recs)

# 5) Example usage
results = get_recommendations(
    terms="cs.CL",
    title="Attention Is All You Need",
    abstract="The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.",
    k=5
)
print(results)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

      score                          terms  \
0  0.773035           ['cs.LG', 'stat.ML']   
1  0.762186  ['cs.LG', 'cs.CL', 'stat.ML']   
2  0.761946           ['cs.LG', 'stat.ML']   
3  0.761946           ['cs.LG', 'stat.ML']   
4  0.761946           ['cs.LG', 'stat.ML']   

                                               title  \
0                            Agglomerative Attention   
1   Augmenting Self-attention with Persistent Memory   
2  I-BERT: Inductive Generalization of Transforme...   
3  I-BERT: Inductive Generalization of Transforme...   
4  I-BERT: Inductive Generalization of Transforme...   

                                            abstract  
0  Neural networks using transformer-based archit...  
1  Transformer networks have lead to important pr...  
2  Self-attention has emerged as a vital componen...  
3  Self-attention has emerged as a vital componen...  
4  Self-attention has emerged as a vital componen...  


In [ ]:
df = pd.DataFrame(results)

In [ ]:
df.head(5)

,score,terms,title,abstract
0,0.773035,"['cs.LG', 'stat.ML']",Agglomerative Attention,Neural networks using transformer-based archit...
1,0.762186,"['cs.LG', 'cs.CL', 'stat.ML']",Augmenting Self-attention with Persistent Memory,Transformer networks have lead to important pr...
2,0.761946,"['cs.LG', 'stat.ML']",I-BERT: Inductive Generalization of Transforme...,Self-attention has emerged as a vital componen...
3,0.761946,"['cs.LG', 'stat.ML']",I-BERT: Inductive Generalization of Transforme...,Self-attention has emerged as a vital componen...
4,0.761946,"['cs.LG', 'stat.ML']",I-BERT: Inductive Generalization of Transforme...,Self-attention has emerged as a vital componen...
